# Setup the APR restraints

In [8]:
import glob as glob
import os as os
import subprocess as sp
import numpy as np

import parmed as pmd
from paprika.restraints import static_DAT_restraint, DAT_restraint

In [2]:
complexes = glob.glob('./OA-G*')

In [4]:
dummy_anchors = [':3', ':4', ':5']
host_anchors  = [':1@AA', ':1@XX', ':1@ZZ']
guest_anchors = [':2@XX', ':2@YY']

attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 '
                '18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 '
              '4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 ' 
              '8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 ' 
              '12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 ' 
              '16.80 17.20 17.60 18.00'
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

In [7]:
static_restraint_atoms = [[dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[2], dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1], host_anchors[2]],
                        ]

static_restraint_distance_fc = 5.0
static_restraint_angle_fc = 100.0

In [6]:
guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]


# Set this from `pytraj`...
guest_restraint_targets = [6.0,
                           180.0,
                           180.0
                          ]
guest_restraint_target_final = [24.0, 180.0, 180.0]
guest_restraint_distance_fc = 5.0
guest_restraint_angle_fc = 100.0

In [ ]:
for complx in complexes:
    
 
    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop',
                       os.path.join(complx, 'AMBER', 'solvate.inpcrd',
                    structure=True)

    static_restraints = []
    for index, atoms in enumerate(static_restraint_atoms):    
        static_DAT_restraint(restraint_mask_list=atoms,
                            num_window_list=windows, 
                            ref_structure=hg,
                            force_constant=static_restraint_angle_fc if len(atoms) > 2 else static_restraint_distance_fc,
                            amber_index=True)
                                    
        static_restraints.append(static_DAT_restraint)
                                    
    guest_restraints = []
    for index, atoms in enumerate(guest_restraint_atoms): 
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = hg
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach['fc_final'] = guest_restraint_angle_fc
        else:
            this.attach['fc_final'] = guest_restraint_distance_fc
        this.attach['target'] = guest_restraint_targets[index]
        this.attach['fraction_list'] = attach_fractions

        this.pull['target_final'] = guest_restraint_target_final[index]
        this.pull['num_windows'] = windows[1]
        this.initialize()
        
        guest_restraints.append(guest_restraint)